# BCC Iron

In this notebook we further analyze the muon sites of Fe to
understand why the local field calculations do not match with the experiment.

First we check the position of the two muon sites that we found, then we recompute local fields.

TODOs are highlighted by 🖍️, check instructions and try yourself!

In [1]:
from pymatgen.core import Structure
from ase.io import read
from ase.atom import Atom
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np
from weas_widget import WeasWidget


scA = read('Supercell_A.cif')
# translate all positions for better visualization
scA.translate([2.,2.,2.])
scA.wrap()

viewer = WeasWidget()
viewer.from_ase(scA)
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=1.)  # 🖍️, TODO: change bond length to show the neighbors of the muon
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7fdf6be45fd0>,))

In [2]:
# Same as above for supercell B
scB = read('Supercell_B.cif')
scB.translate([2.,2.,2.])
scB.wrap()

viewer = WeasWidget()
viewer.from_ase(scB)
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=1.)  # TODO: change me
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7fdf6bfcb390>,))

# Local fields and distortions

In [3]:
# Load muesr modules
import numpy as np
from ase.io import read
from muesr.core.sample import Sample                  # Retains all the sample info.
from muesr.i_o import load_cif
from muesr.utilities import mago_add, print_cell      # For magnetic structure description and visualization
from muesr.utilities import muon_find_equiv            # For finding and including the symmetry equivalent muon positions in the calculation

from muesr.engines.clfc import locfield, find_largest_sphere # Does the sum and returns the local field in its various contributions

np.set_printoptions(suppress=True,precision=5)



In [4]:
# define the sample from the Cif file
fe=Sample()
load_cif(fe, 'Fe.cif')
print_cell(fe)

	 Lattice vectors:
	   a    2.866500000000000    0.000000000000000    0.000000000000000
	   b    0.000000000000000    2.866500000000000    0.000000000000000
	   c    0.000000000000000    0.000000000000000    2.866500000000000
	 Atomic positions (fractional):
	     1 Fe  0.00000000000000  0.00000000000000  0.00000000000000  55.845
	     2 Fe  0.50000000000000  0.50000000000000  0.50000000000000  55.845


True

In [5]:
# Print the status
fe

Sample status: 

 Crystal structure:           Yes
 Magnetic structure:          No
 Muon position(s):            No
 Symmetry data:               Yes

In [6]:
# print the symmetry
fe.sym

Spacegroup(229, setting=1)

In [7]:
# Add one muon in the tetrahedran position ....
fe.add_muon([0.50,0.25,0.0]);

In [8]:
# and find equivalent muon sites
muon_find_equiv(fe);

In [9]:
fe

Sample status: 

 Crystal structure:           Yes
 Magnetic structure:          No
 Muon position(s):            12 site(s)
 Symmetry data:               Yes

# Let's have a look

Let's add the muon positions that we found to the lattice to see the symmetry replica.
Remember the nature of the dipolar field. How many local fields do you expect to see?

In [10]:
viewer = WeasWidget()
iron = read('Fe.cif')
iron.set_array("moment", np.ones(len(iron))) # just for visualization
for muon_position in fe.muons:
    iron.append(Atom('H', iron.cell.cartesian_positions ( muon_position ) ))

viewer.from_ase(iron)
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=0.5)  # TODO: change me
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7fdfc840ca50>,))

# Let's now check the local fields

Local field at muon sites is given by
$$
\mathbf{B} = \mathbf{B}_\text{dip} + \mathbf{B}_\text{hf} + \mathbf{B}_\text{L} + \mathbf{B}_\text{demag} + \mathbf{B}_\text{ext}
$$

Let's compute the various contributions for all the sites.
While doing this also check the dependence of the results on the size of the sphere used to converge the dipolar sum in real space.

One important information about the contact field: the value that we obtain below is simply defined therough a multiplicative factor (ACont) that describes the hyperfine interaction with the nearest neighboring moments. It has no direct physical meaning and is used here to reproduce the result that can be onbtained with accurate DFT simulations.

In [11]:
# We define a new magnetic order 
fe.new_mm() # empty values
fe.mm.k=np.array([0.0,0.0,0.0])                     # set propagation vector
fe.mm.fc= np.array([[0.0+0.j, 0.0+0.j, 2.22+0.j],   # Set fourier components
                     [0.0+0.j, 0.0+0.j, 2.22+0.j]])

In [14]:
# Find the largest radius that we can fit inside a 100x100x100 supercell
# and procede with the Lorentz approach to computer the dipolar contribution.
radius=find_largest_sphere(fe,[100, 100, 100])

#Perform the calculation in real space
r=locfield(fe, 's', [100, 100, 100] ,radius)

B_dip=np.zeros([len(fe.muons),3])
B_Lor=np.zeros([len(fe.muons),3])
B_Cont=np.zeros([len(fe.muons),3])
B_Tot=np.zeros([len(fe.muons),3])

# Store the various contributions
for i in range(len(fe.muons)):
    B_dip[i]=r[i].D # Dipolar
    B_Lor[i]=r[i].L # Lorentz
    r[i].ACont = -0.0644
    B_Cont[i]=r[i].C # Contact
    B_Tot[i]=r[i].T # Total

In [15]:
print("Dipolar Field for all the 12 tetrahedral equivalent sites")
print(B_dip)

# This is and should be same for all the equivalent sites
print("The Lorentz field is {:4.3f} {:4.3f} {:4.3f}".format(*tuple(B_Lor[0])))

print("The contact field is {:4.3f} T".format(np.linalg.norm(B_Cont[0])))

print("Dipolar average of 1 parallel site and 2 antiparallel sites is {:4.5f} T".format(np.linalg.norm(np.sum(B_dip,axis=0))))

print("The sum of the Contact and the Lorentz field is {:4.5f} T".format(np.linalg.norm(B_Lor[0] + B_Cont[0])))

Dipolar Field for all the 12 tetrahedral equivalent sites
[[ 0.     -0.      0.2654]
 [ 0.      0.      0.2654]
 [ 0.     -0.     -0.5308]
 [ 0.     -0.     -0.5308]
 [ 0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.      0.     -0.5308]
 [-0.     -0.     -0.5308]
 [-0.      0.      0.2654]
 [-0.      0.      0.2654]]
The Lorentz field is 0.000 0.000 0.732
The contact field is 1.111 T
Dipolar average of 1 parallel site and 2 antiparallel sites is 0.00000 T
The sum of the Contact and the Lorentz field is 0.37852 T


# So what's going on?

From Solid State Communications, 29, 143 (1979)

    This fact indicates fast diffusion of muon averages out the different magnetic dipolar fields (motional narrowing).

So what should we observe?

The sum of the Lorentz field and the Contact field (and the demagnetization field that depends on the shape of the sample!)